### Tutorial: Getting Music Data with the Last.fm API using Python

### Dataquest 
https://www.dataquest.io/blog/last-fm-api-python/

Lastfm connects with iTunes/Spotify and builds personal profiles for artists

In [4]:
import requests
import json

## Making the request, checkin out the data

FM API account info

Application name	LastFM API practice

API key	7892255aa5636f048b22080765a6fa5e

Shared secret	f8e61223dbf0877f70f429574c545232

Registered to	sagrawal21

In [5]:
api_key = '7892255aa5636f048b22080765a6fa5e'
secret = 'f8e61223dbf0877f70f429574c545232'

In [6]:
USER_AGENT = "Samarth Agrawal"

In [7]:
url='http://ws.audioscrobbler.com/2.0/'

In [8]:
headers = {
    'user-agent': USER_AGENT
}

payload = {
    'api_key': api_key,
    'artist':'Cher',
    'method': 'artist.getTopAlbums',
    'format': 'json'
}

response = requests.get(url, headers=headers, params=payload)

In [9]:
response.status_code

200

In [10]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'http://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = api_key
    payload['format'] = 'json'

    return requests.get(url, headers=headers, params=payload)

In [11]:
top_artists = lastfm_get({
    'method': 'chart.gettopartists'
})

In [12]:
top_artists.status_code

200

In [27]:
import pandas as pd

In [14]:
def j_print(obj):
    #dumps into string
    print(json.dumps(obj, sort_keys=True, indent=4))


In [15]:
type(top_artists.json()) # dict
type(json.dumps(top_artists.json())) # string
data = top_artists.json()

In [16]:
type(data['artists']['artist'])

list

In [17]:
data.keys()

dict_keys(['artists'])

In [18]:
artists = data['artists']
type(artists) #dict
artists.keys()

dict_keys(['artist', '@attr'])

In [19]:
artists['@attr']

{'page': '1', 'perPage': '50', 'totalPages': '63702', 'total': '3185083'}

In [72]:
df = pandas.DataFrame(artists['artist'])

In [77]:
df[df['name']=="Maroon 5"]

,name,playcount,listeners,mbid,url,streamable,image
42,Maroon 5,123973081,3638730,0ab49580-c84f-44d4-875f-d83760ea2cfe,https://www.last.fm/music/Maroon+5,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [78]:
eminem_top_albums = lastfm_get({
    'artist':'eminem',
    'method':'artist.getTopAlbums'
})

In [79]:
eminem_top_albums.json()['topalbums']

{'album': [{'name': 'Recovery',
   'playcount': 28516473,
   'mbid': 'dddf01df-f9f1-4ba6-b414-5ddf1984fc7f',
   'url': 'https://www.last.fm/music/Eminem/Recovery',
   'artist': {'name': 'Eminem',
    'mbid': 'b95ce3ff-3d05-4e87-9e01-c97b66af13d4',
    'url': 'https://www.last.fm/music/Eminem'},
   'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/d15881b09b6041ccad34c2490de618b3.png',
     'size': 'small'},
    {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/d15881b09b6041ccad34c2490de618b3.png',
     'size': 'medium'},
    {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/d15881b09b6041ccad34c2490de618b3.png',
     'size': 'large'},
    {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/d15881b09b6041ccad34c2490de618b3.png',
     'size': 'extralarge'}]},
  {'name': 'The Eminem Show',
   'playcount': 22233279,
   'mbid': 'af71f60c-a8e8-4774-a2b3-30dbfaa13bd6',
   'url': 'https://www.last.fm/music/Eminem/The+Eminem+Show',
   'artist': {'name': 'Eminem',
    '

# Pagination+Rate Limiting

In [84]:
artists['@attr']

{'page': '1', 'perPage': '50', 'totalPages': '63702', 'total': '3185083'}

There are 63k pages and 3 million artists, we could use loop to get all, but need to limit times accessed, or API could lock us out

### First access will take ~ 30 min. For future quick access, store collected data in a cache, easier to access. 

In [2]:
import requests_cache
requests_cache.install_cache()

In [20]:
import time
from IPython.core.display import clear_output


In [30]:
responses = []

page = 1
total_pages = 99999 # this is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'chart.gettopartists',
        'limit': 500,
        'page': page
    }

    # print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # clear the output to make things neater
    clear_output(wait = True)

    # make the API call
    response = lastfm_get(payload)

    # if we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # extract pagination info
    page = int(response.json()['artists']['@attr']['page'])
    total_pages = int(response.json()['artists']['@attr']['totalPages'])

    # append response
    responses.append(response)

    # if it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # increment the page number
    page += 1

Requesting page 6371/6371


In [31]:
r0 = responses[0]
r0_artists = r0.json()['artists']['artist']
r0_df = pd.DataFrame(r0_artists)
r0_df.head()

,name,playcount,listeners,mbid,url,streamable,image
0,The Weeknd,102753233,1480095,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Billie Eilish,57240547,759514,,https://www.last.fm/music/Billie+Eilish,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
2,Dua Lipa,39380079,688910,,https://www.last.fm/music/Dua+Lipa,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Lady Gaga,304012036,3965332,650e7db6-b795-4eb5-a702-5ea2fc46c848,https://www.last.fm/music/Lady+Gaga,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Kanye West,281905661,4556956,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [34]:
r0_df['name'].nunique() # unique

500

In [73]:
frames = [pd.DataFrame(response.json()['artists']['artist']) for response in responses]

In [74]:
all_data=pd.concat(frames)

In [75]:
all_data.index = range(len(all_data.index))
all_data

,name,playcount,listeners,mbid,url,streamable,image
0,The Weeknd,102753233,1480095,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Billie Eilish,57240547,759514,,https://www.last.fm/music/Billie+Eilish,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
2,Dua Lipa,39380079,688910,,https://www.last.fm/music/Dua+Lipa,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Lady Gaga,304012036,3965332,650e7db6-b795-4eb5-a702-5ea2fc46c848,https://www.last.fm/music/Lady+Gaga,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Kanye West,281905661,4556956,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
...,...,...,...,...,...,...,...
9995,McClenney,94665,22431,,https://www.last.fm/music/McClenney,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
9996,Anoushka Shankar,1326534,146782,40ee8fa3-c6d7-4556-b4e2-9f4114565043,https://www.last.fm/music/Anoushka+Shankar,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
9997,Eelke Kleijn,701776,110589,9567a29f-cb0e-4880-8b23-beac15a2cf94,https://www.last.fm/music/Eelke+Kleijn,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
9998,Fatback Band,809927,148328,a7290418-82e2-4016-abf4-3fd3e7db926f,https://www.last.fm/music/Fatback+Band,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [76]:
all_data['name'].nunique()

9996

In [77]:
all_data[all_data['name']=='Kanye West']['url'] #his page

4    https://www.last.fm/music/Kanye+West
Name: url, dtype: object

In [78]:
image1 = all_data[all_data['name']=='Eminem']['image']
type(image1) #series
image1
list(image1)[0][0]['#text']

'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png'

In [79]:
all_data.drop('image', axis=1, inplace=True)
all_data.columns

Index(['name', 'playcount', 'listeners', 'mbid', 'url', 'streamable'], dtype='object')

In [81]:
len_responses = 

,name,playcount,listeners,mbid,url,streamable,image
0,The Weeknd,102753233,1480095,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Billie Eilish,57240547,759514,,https://www.last.fm/music/Billie+Eilish,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
2,Dua Lipa,39380079,688910,,https://www.last.fm/music/Dua+Lipa,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Lady Gaga,304012036,3965332,650e7db6-b795-4eb5-a702-5ea2fc46c848,https://www.last.fm/music/Lady+Gaga,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Kanye West,281905661,4556956,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
...,...,...,...,...,...,...,...
495,Lord Huron,9674903,518986,51c76d97-0635-433c-b26d-66beb81657f8,https://www.last.fm/music/Lord+Huron,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
496,Elliott Smith,105168073,1717908,03ad1736-b7c9-412a-b442-82536d63a5c4,https://www.last.fm/music/Elliott+Smith,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
497,Jay Electronica,3733595,278759,8bf283b9-b1e2-4632-a953-eacd212a543e,https://www.last.fm/music/Jay+Electronica,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
498,Jason Mraz,63474209,2332062,82eb8936-7bf6-4577-8320-a2639465206d,https://www.last.fm/music/Jason+Mraz,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [93]:
len_responses=[len(r.json()['artists']['artist']) for r in responses]
pd.Series(len_responses).value_counts()

0      6351
500      20
dtype: int64

# Only twenty out of 6000 responses returned value!
### That's why 10,000 artists, not 3 million --> but 10k is enough 

# Using other endpoints to augment data

In [99]:
for i in all_data['name']:
    print(alli) 

The Weeknd
Billie Eilish
Dua Lipa
Lady Gaga
Kanye West
Tame Impala
Post Malone
Doja Cat
Ariana Grande
The Beatles
Lana Del Rey
Kendrick Lamar
Childish Gambino
Queen
Radiohead
Drake
Rihanna
Taylor Swift
Gorillaz
Eminem
Arctic Monkeys
David Bowie
Tyler, the Creator
Coldplay
Halsey
Frank Ocean
Harry Styles
Ed Sheeran
Katy Perry
LIL UZI VERT
Beyoncé
BROCKHAMPTON
The Killers
Red Hot Chili Peppers
Calvin Harris
Justin Bieber
Travi$ Scott
Selena Gomez
Lorde
Pink Floyd
Fleetwood Mac
Nirvana
Maroon 5
Miley Cyrus
Mac Miller
The Rolling Stones
Grimes
The Strokes
The Cure
Sam Smith
Daft Punk
Green Day
Khalid
Panic! at the Disco
Britney Spears
Future
Roddy Ricch
Imagine Dragons
Linkin Park
Led Zeppelin
MGMT
Sia
A$AP Rocky
Lauv
The Smiths
Muse
SZA
Paramore
The Chainsmokers
Twenty One Pilots
Camila Cabello
Metallica
Sufjan Stevens
The 1975
David Guetta
Ellie Goulding
Michael Jackson
Madonna
Elton John
Demi Lovato
Lizzo
Shawn Mendes
Charli XCX
Tones And I
Florence + the Machine
Adele
Fall Out Boy
Oasi

Remo Drive
Jadu Heart
Matt Corby
Village People
Stunna 4 Vegas
Air Supply
Mase
D12
Luther Vandross
Todrick Hall
Domo Genesis
Four Year Strong
OH MY GIRL
jackson wang
Rolling Blackouts Coastal Fever
Grace Jones
Danny Elfman
Big Gigantic
Alestorm
Charlotte Day Wilson
weird inside
Tom Grennan
Show Me The Body
Vundabar
Mr Twin Sister
Bobby Womack
Ane Brun
Gustavo Santaolalla
The Big Moon
Dead Can Dance
Glimlip
Dean Blunt
Guustavv
Enigma
The Ronettes
zack fox
Motion City Soundtrack
FloFilz
Tiny Moving Parts
Sophie Ellis-Bextor
Roosevelt
wifisfuneral
Scott BradLee's Postmodern Jukebox
Blue October
Martha Reeves & The Vandellas
Christian French
Wonder Girls
The Durutti Column
The Civil Wars
Alan Silvestri
Tropical Fuck Storm
Epica
GoGo Penguin
Feint
BRENNAN SAVAGE
Minnie Riperton
Black Veil Brides
Hugh Jackman
Kelly Lee Owens
B.B. King
Slash
M|O|O|N
SOHN
Ice Nine Kills
Hamilton Leithauser
Melim
Belly
Slum Village
Ambulo
O Terno
Pretty Lights
Sarah McLachlan
Alceu Valença
The Black Crowes
Pusc

Kensington
鷺巣詩郎
Isaac Delusion
Them
Sonny Rollins
allem iversom
The Raveonettes
Seals & Crofts
Sault
Nonpoint
Siames
Dom Dolla
Wild Child
Vendredi sur Mer
Alexi Murdoch
Howlin' Wolf
Throbbing Gristle
Steve Earle
Austin Wintory
The Saturdays
Wand
Immortal
Georges Bizet
Mitch Murder
ØDYSSEE
Local H
Minus the Bear
The Qemists
ESG
Lunchmoney Lewis
Mc G15
Mata
NERVO
Juniore
iMarkkeyz
Mew
Dungen
frumhere
Damon Albarn
the spirit of the beehive
The Undercover Dream Lovers
Grizfolk
Sega Bodega
Ike & Tina Turner
Turma do Pagode
The Fall of Troy
Equilibrium
Axwell
Romaine LaGrande
Coma
Syd
Lordi
Kailee Morgue
A.C.E
Lambchop
Hovvdy
Maty Noyes
Wild Beasts
Des'ree
Luis Miguel
Mau y Ricky
M-22
Hailu Mergia
Omaure
Zion.T
Dej Loaf
Dierks Bentley
Viktor Vaughn
Tosca
Jonwayne
Roy Ayers
Mgła
Pinkfong
Elvis Depressedly
Mr. Scruff
Benny Goodman
Zeal & Ardor
Arin Ray
Ball Park Music
Big Black Delta
DJ Seinfeld
Ann Peebles
King Dude
Exaltasamba
Hatchie
Maverick Sabre
Pogo
Bluedoom
Tokio Hotel
Ben Folds Five
G

The Gathering
Josh Groban
Caro
Antimatter
Chiodos
brillion.
Fred V & Grafix
Barrington Levy
The Murder Capital
Onyx
Gioacchino Rossini
John Barry
Propagandhi
Corrosion of Conformity
Richard Ashcroft
JUNGLEPUSSY
Little Boots
The Beautiful South
Luciano Pavarotti
That Poppy
In Hearts Wake
Laid Back
Charlie Heat
Max Romeo
Mc Fioti
Crystal Lake
Gábor Szabó
Gin Wigmore
Marc Cohn
Grinderman
Maximum the Hormone
Tyler Ward
Sewerslvt
Marina & the Diamonds
Archy Marshall
SEGA Sound Team
Brutus
Johnny Nash
Harold Melvin & The Blue Notes
LVNDSCAPE
Modestep
Alela Diane
Jay1
Our Lady Peace
Amanda Lee
Sylosis
Blossom Dearie
Girl Ray
Thank You Scientist
John Fahey
Calibre
Mc Don Juan
The Whispers
Currents
Mounika.
Jerry Folk
Tarja
I the Mighty
Daniel Pemberton
Nico Santos
Vader
The Delfonics
Sid
From Ashes to New
Fleshgod Apocalypse
Pelican
Samsa
And So I Watch You From Afar
Abakus
Skinny Puppy
Oliver
Sultan + Shepard
Grum
Seekae
Ce Ce Peniston
Pat Metheny Group
Hoody
The Coasters
Ferrugem
Alex Lahey


Djrum
Brooks
Clara Mae
Ryan
Tobacco
Spencer Brown
Dschinghis Khan
дора
Melissa Etheridge
Eli Young Band
Cartel
Tides From Nebula
Sean Nicholas Savage
Deeb
Conway Twitty
Yasuaki Shimizu
Flo Morrissey
Flunk
Younger Brother
F.O.O.L
Aaron
Chris Webby
YUNG BUDA
Millie Turner
9MUSES A
Los Prisioneros
The Brobecks
Rebelution
Jamie Lawson
Ruck P
Mountain Man
Kosheen
Kadavar
Vini Vici
METZ
Adam Hicks
Lewis OfMan
Casper
Alphabeat
Loggins & Messina
Soulchef
Donnie & Joe Emerson
GAYAZOV$ BROTHER$
Transplants
Bootsy Collins
Cory Wong
SOJA
BØJET
Rizzle Kicks
Alexander Panagopoulos
James Supercave
Wanda Jackson
Funky DL
Panda Eyes
Bomb the Music Industry!
Mushroomhead
Mark Pritchard
Destroy Boys
Hank Williams Jr.
Life Without Buildings
Rincon Sapiência
Finch Asozial
Laraaji
Touch Sensitive
Keem The Cipher
Alexandra Zvereca
Alesana
Paul Hardcastle
Luxury Elite
bohnes
Tsuko G.
Mojave 3
Nevermore
Tiziano Ferro
Sam Gendel
Loretta Lynn
Ginger Root
Mychael Danna
Kenny Burrell
Ghali
Not3s
Beth Orton
Besomor

Shawn Lee's Ping Pong Orchestra
La Bien Querida
Agni Bjark
Viscera
Elvenking
布袋寅泰
Black Strobe
Boy Azooga
Avocuddle
We Butter the Bread With Butter
Anthem Lights
Luca Hänni
Christopher Dicker
Pete and the Pirates
The Equatics
Rage
Orbit Culture
Delle Alpi
Maurice Fulton
Vanilla Fudge
tonebox
Alex Lustig
Snow
Veorra
Win and Woo
Don Caballero
DJ Zinc
Harry James
Kelly Moran
Mildlife
T.R.U.
El Cuarteto De Nos
John Mayall & The Bluesbreakers
Егор Летов
Nathy Peluso
Aleksy Wysocki
Asobi Seksu
Masterplan
Quadeca
Shiloh Dynasty
Real McCoy
Herve Pagez
Elettra Lamborghini
T.S.O.L.
Nimo
Mating Ritual
Closure in Moscow
Nuages
whitewoods
Angerfist
Paul Revere & The Raiders
Jaira burns
Always Never
Off With Their Heads
Hare Squead
Azumi Inoue
Hogland
Friday Night Plans
June
Binaural Landscapes
Mad Caddies
Gotthard
4 Strings
The Association
Amelie Lens
Aksel
Laurel
All-4-One
Sheer Mag
Moy Finneas
Augustus Pablo
Yüzyüzeyken Konuşuruz
Mc TH
Planet Hemp
Juultje Berben
Reba McEntire
Indeep
Total Giovann

In [109]:
all_data.set_index('name', inplace=True)

In [114]:
all_data['url']

name
The Weeknd                https://www.last.fm/music/The+Weeknd
Billie Eilish          https://www.last.fm/music/Billie+Eilish
Dua Lipa                    https://www.last.fm/music/Dua+Lipa
Lady Gaga                  https://www.last.fm/music/Lady+Gaga
Kanye West                https://www.last.fm/music/Kanye+West
                                       ...                    
McClenney                  https://www.last.fm/music/McClenney
Anoushka Shankar    https://www.last.fm/music/Anoushka+Shankar
Eelke Kleijn            https://www.last.fm/music/Eelke+Kleijn
Fatback Band            https://www.last.fm/music/Fatback+Band
Tom Gregory              https://www.last.fm/music/Tom+Gregory
Name: url, Length: 10000, dtype: object

In [141]:
def get_tags(name):
    payload = {
        'method': 'artist.getTopTags',
        'artist': name,
        'api_key':api_key,
        'format':'json',
    }

    headers = {'user-agent': USER_AGENT}
    url = 'http://ws.audioscrobbler.com/2.0/'
    
    rep= requests.get(url, headers=headers, params=payload).json()
    tags = rep['toptags']['tag'][:3]
    s=""
    for tag in tags:
       s+= tag['name'] + " "
    return s
    

In [142]:
get_tags("Billie Eilish")

'pop indie pop electronic '